# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

In [1]:
! pip install pandas
! pip install regex
! pip install matplotlib
! pip install nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 134.2 MB/s eta 0:00:00m eta 0:00:010:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.5/503.5 kB 107.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 131.3 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.2/768.2 kB 23.8 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 134.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 944.1/944.1 kB 119.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 148.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 120.9 MB/s eta 0:00:00

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, ru

In [2]:
! pip install sklearn

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.2/31.2 MB 45.4 MB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 36.8 MB/s eta 0:00:00m eta 0:00:010:01:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1304 sha256=e13334de667db3fbe0889333ba290ca2cf41c86cc42aeadc2a2d4ffef7fab743
  Stored in directory: /home/gitpod/.cache/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [3]:

import pandas as pd 
import regex as reg
import re
import matplotlib.pyplot as plt
import unicodedata
import nltk
import string
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

from sklearn import model_selection, svm
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords



In [4]:
df_raw = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/NLP-project-tutorial/main/url_spam.csv')

In [5]:
df_raw.info

<bound method DataFrame.info of                                                     url  is_spam
0     https://briefingday.us8.list-manage.com/unsubs...     True
1                                https://www.hvper.com/     True
2                    https://briefingday.com/m/v4n3i4f3     True
3      https://briefingday.com/n/20200618/m#commentform    False
4                           https://briefingday.com/fan     True
...                                                 ...      ...
2994  https://www.smartcitiesworld.net/news/news/dee...    False
2995                      https://www.youtube.com/watch     True
2996  https://techcrunch.com/2019/07/04/an-optimisti...    False
2997  https://www.technologyreview.com/2019/12/20/13...    False
2998       https://www.bbc.com/news/technology-51018758    False

[2999 rows x 2 columns]>

In [6]:
#Vamos a pasar a 0 y 1 la variable objetivo

df_raw['is_spam'] = df_raw['is_spam'].apply(lambda x: 1 if x == True else 0)

In [7]:
df_raw.info

<bound method DataFrame.info of                                                     url  is_spam
0     https://briefingday.us8.list-manage.com/unsubs...        1
1                                https://www.hvper.com/        1
2                    https://briefingday.com/m/v4n3i4f3        1
3      https://briefingday.com/n/20200618/m#commentform        0
4                           https://briefingday.com/fan        1
...                                                 ...      ...
2994  https://www.smartcitiesworld.net/news/news/dee...        0
2995                      https://www.youtube.com/watch        1
2996  https://techcrunch.com/2019/07/04/an-optimisti...        0
2997  https://www.technologyreview.com/2019/12/20/13...        0
2998       https://www.bbc.com/news/technology-51018758        0

[2999 rows x 2 columns]>

In [8]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   url      2999 non-null   object
 1   is_spam  2999 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


In [9]:
df_raw['url'].sample(20)

2347    https://www.cnbc.com/2020/06/26/this-chart-sho...
2233    https://www.wsj.com/articles/amazon-to-keep-mo...
2167    https://narratively.com/conscripted-into-the-e...
1089               https://vincerowatches.com/pages/skimm
1081    https://docs.google.com/forms/d/e/1FAIpQLSdkbW...
653     https://www.fool.com/the-ascent/credit-cards/l...
2721    https://www.axios.com/hong-kong-national-secur...
226     https://www.cnbc.com/2020/06/19/us-treasury-sb...
634     https://www.businessinsider.com/amy-mcgrath-vs...
1241    https://huckberry.com/store/looft/category/p/6...
51      https://www.caltech.edu/about/news/natural-flu...
243     https://www.politico.com/news/2020/06/21/takea...
2769    https://time.com/5791661/who-coronavirus-pande...
1308    https://www.wired.co.uk/article/door-to-door-s...
910     https://www.travelchannel.com/shows/anthony-bo...
2824    https://numlock.substack.com/p/numlock-sunday-...
305                       https://www.rentalcarrally.com/
2337    https:

In [10]:
df_raw['is_spam'].value_counts()

0    2303
1     696
Name: is_spam, dtype: int64

In [11]:
# Vamos a ver como esta nuestra variable objetivo en este caso es is_spam
df_raw['is_spam'].value_counts()
print("Hay spam: ",len(df_raw.loc[df_raw.is_spam==1]))
print("Que no son spam : ",len(df_raw.loc[df_raw.is_spam==0]))
print(df_raw.shape)


Hay spam:  696
Que no son spam :  2303
(2999, 2)


In [12]:
# Eliminar los duplicados

df_raw = df_raw.drop_duplicates()
df_raw = df_raw.reset_index(inplace = False)[['url','is_spam']]
df_raw.shape

(2369, 2)

Empezamos el proceso de Limipieza de los datos

In [13]:
# colocar todos los textos en minusculas
df_raw['url'] = df_raw['url'].str.lower()

In [14]:
nltk.download('stopwords')
nltk.download('punkt')

stopwords = nltk.corpus.stopwords.words('english')
punctuation = string.punctuation

print(stopwords[:5])
print(punctuation)

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['i', 'me', 'my', 'myself', 'we']
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [15]:
df_aux = df_raw.copy()

In [16]:

# Proceso de limpieza de datos

limpieza = []

for p in range(len(df_aux.url)):
    desc = df_aux['url'][p]
    
    #savar la puntuacion
    desc = re.sub('[^a-zA-Z]', ' ', desc)
    
    #borrar etiquetas especiales
    desc=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",desc)
    
    #borrar digitos y caracteres especiales
    desc=re.sub("(\\d|\\W)+"," ",desc)
    
    limpieza.append(desc)

#assign the cleaned descriptions to the data frame
df_aux['url'] = limpieza
        
df_aux.head()

url  is_spam
0  https briefingday us list manage com unsubscribe        1
1                              https www hvper com         1
2                  https briefingday com m v n i f         1
3             https briefingday com n m commentform        0
4                         https briefingday com fan        1

In [17]:
df_aux['url'].str.split(expand=True).stack().value_counts()[:60]

https          2336
com            2065
www            1512
the             354
html            296
news            274
a               252
us              248
to              218
of              173
coronavirus     172
e               150
org             146
c               136
article         131
b               124
in              115
and             113
morningbrew     105
story           105
nytimes         101
on              101
daily            99
d                98
stories          94
utm              90
for              90
youtube          89
v                89
trump            88
numlock          87
watch            86
f                78
is               77
new              76
p                69
world            68
substack         68
reuters          65
covid            63
s                62
briefingday      61
index            61
vox              59
en               59
articles         58
cnn              58
iduskbn          58
politics         56
co               56


In [18]:
# Creamos una lista de stopwords
stop_words = ['http','www','com','you','your','for','not','have','is','in','im','from','to','https','e','c','v','b','f','p']

In [19]:

def remove_stopwords(message):
  if message is not None:
    words = message.strip().split()
    words_filtered = []
    for word in words:
      if word not in stop_words:
        words_filtered.append(word) 
    result = " ".join(words_filtered)         
  else:
    result = None

  return result 


In [20]:
df_aux['url']=df_aux['url'].apply(remove_stopwords)

In [21]:
df_aux.sample(10)

url  is_spam
967               longreads the long con of britishness        0
1632  cidrap umn edu news perspective covid cases am...        0
911   people country dixie chicks change name the ch...        0
565   axios trump coronavirus testing kidding a a ab...        0
341   airspacemag history of flight first air force one        0
1282                 annfriedman us list manage profile        1
1479           youtube channel ucyan mg u cjy zi ikwaug        1
1464  chicagobusiness airlines airports american air...        0
742   morningbrew daily stories bayer agrees billion...        0
2245  numlock substack numlock sunday ali griswold o...        1

In [22]:
df_aux['url'].str.split(expand=True).stack().value_counts()[:60]

the            354
html           296
news           274
a              252
us             248
of             173
coronavirus    172
org            146
article        131
and            113
story          105
morningbrew    105
on             101
nytimes        101
daily           99
d               98
stories         94
utm             90
youtube         89
trump           88
numlock         87
watch           86
new             76
world           68
substack        68
reuters         65
covid           63
s               62
index           61
briefingday     61
vox             59
en              59
cnn             58
iduskbn         58
articles        58
politics        56
co              56
n               54
cnbc            54
sunday          51
business        49
court           48
apnews          47
u               46
email           46
facebook        46
health          45
be              41
supreme         41
bbc             41
blog            40
are             40
black       

Esto es algo que estaba en el material de estudio para poder probar como funciona

df['len_url'] = df['url'].apply(lambda x : len(x))
df['contains_subscribe'] = df['url'].apply(lambda x : 1 if "subscribe" in x else 0)
df['contains_hash'] = df['url'].apply(lambda x : 1 if "#" in x else 0)
df['num_digits'] = df['url'].apply(lambda x : len("".join(_ for _ in x if _.isdigit())) )
df['non_https'] = df['url'].apply(lambda x : 1 if "https" in x else 0)
df['num_words'] = df['url'].apply(lambda x : len(x.split("/")))

target = 'is_spam'
features = [f for f in df.columns if f not in ["url", target]]
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=0)

In [23]:
# Ahora que hemos limpiado nuestros datos
# procedemos a hacer una copia con el dataset a trabajar en el final
df = df_aux.copy()

In [24]:
X = df['url']
y = df['is_spam']
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,random_state=121)

In [25]:
# Vectorizador
vec = CountVectorizer(stop_words='english')
X_train = vec.fit_transform(X_train).toarray()
X_test = vec.transform(X_test).toarray()

In [26]:
print(X_train)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [27]:
X_train.shape

(1776, 4825)

In [28]:
X_test.shape

(593, 4825)

In [29]:
print(vec.get_feature_names_out())

['aa' 'aab' 'aaron' ... 'zulalimtm' 'zwift' 'zwn']


In [30]:
nb = MultinomialNB()

In [31]:
nb.fit(X_train, y_train)

MultinomialNB()

In [32]:
nb.score(X_train,y_train)

0.956081081081081

In [33]:
nb.score(X_test,y_test)

0.9173693086003373

In [34]:
predictions = nb.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.93      0.98      0.96       532
           1       0.68      0.38      0.48        61

    accuracy                           0.92       593
   macro avg       0.80      0.68      0.72       593
weighted avg       0.91      0.92      0.91       593



In [35]:
#Creando la Matriz

message_vectorizer = CountVectorizer().fit_transform(df['url'])

# Haciendo el split de los datos

X_train, X_test, y_train, y_test = train_test_split(message_vectorizer, df['is_spam'], test_size = 0.40, random_state = 121, shuffle = True)

In [36]:
cl = svm.SVC(C=1.0, kernel='linear', degree=4, gamma='auto')

In [37]:
cl.fit(X_train, y_train)
pred = cl.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.95      0.98      0.96       847
           1       0.72      0.54      0.62       101

    accuracy                           0.93       948
   macro avg       0.84      0.76      0.79       948
weighted avg       0.92      0.93      0.92       948



In [38]:
pickle.dump(cl, open('../models/texto_NLP.pkl', 'wb'))

In [39]:
# Si queremos cargar el archivo guardado en la carpeta models

load_model = pickle.load(open('../models/texto_NLP.pkl', 'rb'))

In [41]:
load_model

SVC(degree=4, gamma='auto', kernel='linear')